<!-- (evaluation)= -->
# WeatherBench 2 Evaluation Quickstart

<a target="_blank" href="https://colab.research.google.com/github/google-research/weatherbench2/blob/main/docs/source/evaluation.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this notebook, we will cover the basic functionality of the WeatherBench evaluation framework.

The WeatherBench evaluation framework takes two datasets for forecast and ground truth (called obs, even though reanalysis datasets like ERA5 are not observations), computes and saves the specified metrics.

Here, we will evalute ECMWF's HRES forecast against ERA5.

# 新段落

In [1]:
# Pip might complain about the Pandas version. The notebook should still work as expected.
!pip install git+https://github.com/google-research/weatherbench2.git

  Cloning https://github.com/google-research/weatherbench2.git to /tmp/pip-req-build-c0safvef
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/weatherbench2.git /tmp/pip-req-build-c0safvef
  Resolved https://github.com/google-research/weatherbench2.git to commit b0dc18dee8ba2c106a80e6a020d052e19f2d2c1b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... d

In [1]:
import apache_beam   # Needs to be imported separately to avoid TypingError
import weatherbench2
import xarray as xr

In [2]:
# Run the code below to access cloud data on Colab!
from google.colab import auth
auth.authenticate_user()

### Specify input datasets

Let's take a look at the datasets. Currently, the WeatherBench pipeline requires all input dataset to be stored as Zarr files.

In [3]:
forecast_path = 'gs://weatherbench2/datasets/hres/2016-2022-0012-64x32_equiangular_conservative.zarr'
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'
climatology_path = 'gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_conservative.zarr'

Generally, we follow ECMWF's naming conventions for the input files.

* `time` [np.datetime64]: Time at forecast is initialized
* `lead_time` or `prediction_timedelta` [np.timedelta64]: Lead time
* `latitude` [float]: Latitudes from -90 to 90
* `longitude` [float]: Longitudes from 0 to 360
* `level` [hPa]: Pressure levels (optional)

We don't actually need to open the forecast and obs datasets at this point, but we will do so here to see their structure.

#### forecast_path高分辨率确定性预报 数据介绍
它是谁：ECMWF 的HRES（高分辨率确定性预报），作为一个强力物理模型基线，供你和 AI 模型 PK。
时间轴：起报时间来自每天 00Z 和 12Z（文件名里的 0012 就是这个意思），覆盖 2016–2022。每个起报有一串预报时效（prediction_timedelta）。官方示例显示 prediction_timedelta 有 41 个点、6 小时步长，从 0h 到 240h（含 0h），也就是 10 天预报序列。
WeatherBench 2

空间网格：64x32_equiangular_conservative 表示被重采样到 5.625° 等经纬网格（64×32），用一阶保体积的保守重网格方法；64×32 版本不含两极（保证经纬均匀）。
WeatherBench 2

变量与层次：与 ERA5 对齐的13 个等压面（50–1000 hPa）和常见单层变量（如 10m 风、2m 气温、海平面气压、位势高度、u/v 风、比湿等），命名遵循 ECMWF/WB2 习惯（如 geopotential, u_component_of_wind）。
WeatherBench 2

评测里的角色：这是**“你的预测”的替身/对照：把它喂给 WB2，和 obs（ERA5）对比，能得到 RMSE、ACC 等分数；你后面把这条路径替换成你们 Diffusion+Transformer 的 Zarr，就能与 HRES 同框对比**。

In [4]:
xr.open_zarr(forecast_path)

/tmp/ipython-input-2213774576.py:1: FutureWarning: In a future version, xarray will not decode the variable 'prediction_timedelta' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  xr.open_zarr(forecast_path)


<xarray.Dataset> Size: 172GB
Dimensions:                   (time: 5114, prediction_timedelta: 41,
                               longitude: 64, latitude: 32, level: 13)
Coordinates:
  * time                      (time) datetime64[ns] 41kB 2016-01-01 ... 2022-...
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 328B 00:...
  * longitude                 (longitude) float64 512B 0.0 5.625 ... 348.8 354.4
  * latitude                  (latitude) float64 256B -87.19 -81.56 ... 87.19
  * level                     (level) int32 52B 50 100 150 200 ... 850 925 1000
Data variables: (12/16)
    10m_u_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    10m_wind_speed            (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    2m_temperature            (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    geopotential              (time, prediction_timedelta, level, longitude, latitude) float32 22GB dask.array<chunksize=(4, 1, 13, 64, 32), meta=np.ndarray>
    mean_sea_level_pressure   (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    ...                        ...
    total_precipitation_24hr  (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    total_precipitation_6hr   (time, prediction_timedelta, longitude, latitude) float32 2GB dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    u_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 22GB dask.array<chunksize=(4, 1, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 22GB dask.array<chunksize=(4, 1, 13, 64, 32), meta=np.ndarray>
    vertical_velocity         (time, prediction_timedelta, level, longitude, latitude) float32 22GB dask.array<chunksize=(4, 1, 13, 64, 32), meta=np.ndarray>
    wind_speed                (time, prediction_timedelta, level, longitude, latitude) float32 22GB dask.array<chunksize=(4, 1, 13, 64, 32), meta=np.ndarray>

#### obs_path 数据介绍

它是谁：ERA5 再分析数据——WB2 的真值（ground truth / observations）。
时间轴：WB2 的 ERA5 提供 1959 年起、6 小时时间分辨率；你看到这条是到 2022 年的 64×32 版本（桶里也有更长到 2023-01-10 的新版，保留旧版是为了延续性）。评测通常选 2020 年做窗口。
WeatherBench 2

空间网格：同样是 64×32 等经纬、保守重网格到目标分辨率，便于与 HRES/你们模型可比。
WeatherBench 2

变量与层次：13 个等压面 + 多个单层变量，且包含一批已计算好的派生变量（如风速、相对湿度等），变量名与 HRES 一致或可映射。
WeatherBench 2

评测里的角色：这是对照真值。WB2 会逐 lead、逐区域把 forecast（上面的 HRES 或你们模型预测）与这里的 ERA5 做逐格/聚合对比并产出分数

In [5]:
xr.open_zarr(obs_path)

<xarray.Dataset> Size: 81GB
Dimensions:                                           (time: 92044,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * time                                              (time) datetime64[ns] 736kB ...
  * longitude                                         (longitude) float64 512B ...
  * latitude                                          (latitude) float64 256B ...
  * level                                             (level) int64 104B 50 ....
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>

Some metrics like the ACC also require a climatology file. Refer to the paper for details on how this is computed.

#### ERA5 的“气候态”（climatology） 数据介绍
它是谁：ERA5 的“气候态”（climatology）——按年中第几天 × 一天中的四个 6h 时刻（0/6/12/18 UTC）做的平滑气候平均场，用于“距平”相关等指标。
时间/维度结构：不是“逐日序列”，而是按dayofyear × hour × (lat, lon) × (可选 level)存储的“季节循环模板”；WB2 提供 1990–2017 与 1990–2019 两个版本，你这里选的是 1990–2019、6 小时分辨率 的 64×32 版本。
WeatherBench 2

为什么需要它：很多“距平”指标（最典型是 ACC = Anomaly Correlation Coefficient）需要把预测和真值减去气候态再比较，这样能公平评估环流“相位/形势是否对”，不被季节均值淹没；SEEPS（降水评分）等也会用到气候分布信息。没有 climatology，你的 ACC 就算不出来。
WeatherBench 2

空间网格与方法：同样是 64×32 等经纬、保守重网格一致化。


In [6]:
climatology = xr.open_zarr(climatology_path)
climatology

<xarray.Dataset> Size: 3GB
Dimensions:                                         (hour: 4, dayofyear: 366,
                                                     longitude: 64,
                                                     latitude: 32, level: 13)
Coordinates:
  * hour                                            (hour) int64 32B 0 6 12 18
  * dayofyear                                       (dayofyear) int64 3kB 1 ....
  * longitude                                       (longitude) float64 512B ...
  * latitude                                        (latitude) float64 256B -...
  * level                                           (level) int64 104B 50 ......
Data variables: (12/53)
    10m_u_component_of_wind                         (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                         (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    10m_wind_speed                                  (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    2m_dewpoint_temperature                         (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    2m_temperature                                  (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    above_ground                                    (hour, dayofyear, level, longitude, latitude) float32 156MB dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>
    ...                                              ...
    volumetric_soil_water_layer_1                   (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    volumetric_soil_water_layer_2                   (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    volumetric_soil_water_layer_3                   (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    volumetric_soil_water_layer_4                   (hour, dayofyear, longitude, latitude) float32 12MB dask.array<chunksize=(4, 366, 64, 32), meta=np.ndarray>
    vorticity                                       (hour, dayofyear, level, longitude, latitude) float32 156MB dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>
    wind_speed                                      (hour, dayofyear, level, longitude, latitude) float32 156MB dask.array<chunksize=(4, 366, 13, 64, 32), meta=np.ndarray>

forecast（HRES 或你们的模型输出） vs obs（ERA5 真值） → 计算 RMSE/MAE/Bias/ACC 等分数；

climatology（气候态） → 提供“去季节均值”的参考，用来算 ACC/SEEPS 等需要距平或分位信息的指标；

三者网格一致（或先用 WB2 的 regrid 保守重网格到同一分辨率），时间约定默认用 by-init（按起报组织样本），确保和官网可比。
WeatherBench 2

---


### Set up WeatherBench configuration

Next, we will define a bunch of configuration instances to specify exactly what we want to evaluate.

In [7]:
from weatherbench2 import config

#### Data configuration

The file paths are defined in a Paths config object, alongside an output directory:

这个对象专门用来定义所有相关的文件和文件夹路径。

forecast=forecast_path：这是你模型生成的预测数据文件路径（.zarr 格式）。

obs=obs_path：这是观测数据或“基准真相”（Ground Truth，通常是 ERA5）的文件路径（.zarr 格式）。

output_dir='./'：这是一个输出目录，用来指定评估完成后，保存结果（例如 ... .nc 文件）的位置。

In [8]:
paths = config.Paths(
    forecast=forecast_path,
    obs=obs_path,
    output_dir='./',   # Directory to save evaluation results
)

In addition, we specify a Selection object that selects the variables and time period to be evaluated.

这个对象非常关键，它用来指定你要评估的具体数据子集。这可以确保你只在关心的变量、时间和高度层上进行评估。

variables=['geopotential', '2m_temperature']：指定一个列表，包含你希望评估的天气变量名称。在这个例子中，你只想评估“geopotential”（位势高度）和“2m_temperature”（2米温度）。

levels=[500, 700, 850]：指定要评估的气压层（单位是 hPa）。这对于像 'geopotential' 这样的多层变量是必需的。评估将只针对这三个高度层进行。

time_slice=slice('2020-01-01', '2020-12-31')：定义评估的时间范围。这里指定了从 2020 年 1 月 1 日到 2020 年 12 月 31 日的整整一年。

In [9]:
selection = config.Selection(
    variables=[
        'geopotential',
        '2m_temperature'
    ],
    levels=[500, 700, 850],
    time_slice=slice('2020-01-01', '2020-12-31'),
)

Together they make up the Data config:

In [10]:
data_config = config.Data(selection=selection, paths=paths)

----


#### Evaluation configuration

Next, we can defined which evaluation we want to run. To do so, we can define a dictionary of `config.Eval`s, each of which will be evaluated separately and saved to a different file. Eval instances contain the metrics objects, defined in metrics.py.

Note that for ACC, we additionally need to pass the climatology opened earlier.

eval_configs 是一个字典，其中：

键 (Key)（例如 'deterministic'）：是你为这组评估起的名字。评估结果会保存在一个以此命名的文件中（例如 deterministic.nc）。

值 (Value)（config.Eval(...)）：是一个 config.Eval 对象，它打包了所有评估的设置

metrics 是一个字典，定义了你想要计算的所有评估指标。

'mse': MSE()：计算均方误差 (Mean Squared Error)。

'acc': ACC(climatology=climatology)：计算异常相关系数 (Anomaly Correlation Coefficient)。

注意：正如图片中的文字提示，计算 ACC 时，你需要额外传入一个 climatology (气候平均态) 数据集。

In [11]:
from weatherbench2.metrics import MSE, ACC

eval_configs = {
  'deterministic': config.Eval(
      metrics={
          'mse': MSE(),
          'acc': ACC(climatology=climatology)
      },
  )
}

The evaluation configs also have an option to evaluate **particular regions**, such as a geographical lat-lon box. These are defined as region objects defined in regions.py. All regions will be evaluated separately and saved as an additional dimension in the dataset. If no region is specified, evaluation will be done globally.

regions 也是一个字典，定义了你希望在哪些特定的地理区域上分别计算上述指标。

如果你不提供 regions 参数（如第一个例子所示），评估将默认在“全球” (global) 范围进行。

如果你提供了 regions 参数（如第二个例子所示），评估将会对你定义的每一个区域都单独计算一遍指标。

图片中的例子定义了三个区域：

'global': SliceRegion()：代表全球范围。

'tropics': SliceRegion(lat_slice=slice(-20, 20))：定义了一个“热带”区域，范围是南纬20度到北纬20度。

'extra-tropics': ExtraTropicalRegion()：使用一个预定义的类来代表“热带以外”的区域。

In [12]:
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion

regions = {
    'global': SliceRegion(),
    'tropics': SliceRegion(lat_slice=slice(-20, 20)),
    'extra-tropics': ExtraTropicalRegion(),
}

eval_configs = {
  'deterministic': config.Eval(
      metrics={
          'mse': MSE(),
          'acc': ACC(climatology=climatology)
      },
      regions=regions
  )
}

### Evaluate

Now, we are already done and can run the evaluation. We can do so in memory for smaller datasets or as a Beam pipeline (recommended for anything larger than 64x32 resolution).

In [13]:
from weatherbench2.evaluation import evaluate_in_memory, evaluate_with_beam

In [14]:
evaluate_in_memory(data_config, eval_configs)   # Takes around 5 minutes

/usr/local/lib/python3.12/dist-packages/weatherbench2/evaluation.py:117: FutureWarning: In a future version, xarray will not decode the variable 'prediction_timedelta' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  forecast = xr.open_zarr(
/usr/local/lib/python3.12/dist-packages/weatherbench2/evaluation.py:67: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if not 

KeyboardInterrupt: 

For the Beam version, evaluation is split across chunks. The chunk sizes can be specified alongside the Beam runner. For more detail on running Beam in a distributed fashion on the cloud, see [this guide](https://weatherbench2.readthedocs.io/en/latest/beam-in-the-cloud.html).

In [17]:
evaluation.evaluate_with_beam(
     data_config,
     eval_configs,
     runner='DirectRunner',
     input_chunks={'time': 20},
 )

/usr/local/lib/python3.12/dist-packages/weatherbench2/evaluation.py:117: FutureWarning: In a future version, xarray will not decode the variable 'prediction_timedelta' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  forecast = xr.open_zarr(
/usr/local/lib/python3.12/dist-packages/weatherbench2/evaluation.py:67: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if not 

ValueError: chunks key 'time' is not a dimension on the provided dataset(s)

### Results

Results are saved as NetCDF files under the name of the eval config.

In [ ]:
results = xr.open_dataset('./deterministic.nc')
results

#### 2) deterministic.nc 的结构长啥样？

文档直接展示了典型结构（只评了 mse 与 acc 两个指标、三个区域、三个等压面）：

坐标（Coordinates）：

lead_time: 0h, 6h, …, 240h（共 41 个 6 小时时效）；

region: global, tropics, extra-tropics（你在 eval 里配的区域会出现在这里）；

level: 500/700/850（只有对有等压层的变量才有这维）；

metric: 这次是 acc, mse（你配了什么指标，这里就列什么）。

数据变量（Data variables）：每个气象量一个变量：

例如 geopotential 的维度是 (metric, region, lead_time, level)；

2m_temperature 没有 level 维，所以是 (metric, region, lead_time)。

维度大小：lead_time:41, region:3, level:3, metric:2（与你选择的指标/区域/层一致）。
WeatherBench 2

一句话：整份文件就是“各指标 × 各区域 × 各时效（× 各等压面）”的成绩表，按变量分开存。

Note that to compute the RMSE, we follow ECMWF's convention by taking the square root after the time mean. To do this in WB2, first compute the MSE and then take the square root of the saved MSE results.

In [ ]:
results = xr.concat(
    [
    results,
    results.sel(metric=['mse']).assign_coords(metric=['rmse']) ** 0.5
    ],
    dim='metric'
)

In [ ]:
results['geopotential'].sel(metric='rmse', level=500, region='global').plot();

### Next steps

This quickstart guide shows the basic functionality of the WeatherBench evaluation code but there is more to explore.

For running evaluation from the command line, see [this guide](https://weatherbench2.readthedocs.io/en/latest/command-line-scripts.html). For a complete overview of the entire evaluation workflow, check out the ["submission" guide](https://weatherbench2.readthedocs.io/en/latest/submit.html).